<a href="https://colab.research.google.com/github/emerenan/xlsx_validation/blob/main/ie_validations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [74]:
import pandas as pd
import numpy as np
import datetime as dt
from datetime import datetime
import re
from openpyxl import Workbook, styles
from openpyxl.styles import PatternFill, Font
from openpyxl.styles.differential import DifferentialStyle
from openpyxl.formatting.rule import Rule

def read_files(path, sheetname, n_skiprows, n_skip_columns, site_index):
    """
    Params:\n
    path: parth of file in the computer.\n
    n_skiprows: Number of rows to delete in the original file,.\n
    columns_to_convert: Columns to convert the data general type. \n
    n_skipcolumn: Columns to skip in the original file. \n
    endrow = pass 0 to read everything, 1 to count entire
    columns_order: List of columns names in specific order to pass in the engine.\n
    """
    df = pd.read_excel(path, sheet_name = sheetname, skiprows = n_skiprows)

    df = df.iloc[:,n_skip_columns:]

    # define the entiry columns which doesn't have NaN 
    df = df.dropna(subset=[site_index], axis=0)
    """cont = 0
    for i in df.iloc[:,site_col]:
        if pd.isnull(i):
            break # acertar o break
        else:
            cont +=1
    df = df.iloc[:cont, :]
    #df.columns = columns_order"""
    
    # convert intery columns to integer 
    #df[columns_integer_convert] = df[columns_integer_convert].fillna(0)
    #df[columns_integer_convert] = df[columns_integer_convert].astype('int64')

    return df

def lower_str(columns):
    newlist = list(map(lambda x: x.lower(), columns))
    return newlist

def check_df(df, error_msg):
    if df == None or df.empty:
        return f'{error_msg}'
    else:
        return df

def count_duplicates(lista):
    count_dict = {}
    for entry in lista:
        if entry in count_dict.keys():
            count_dict[entry] += 1
        else:
            count_dict[entry] = 1
    
    duplicates = {}
    for k, v in count_dict.items():
        if v > 1:
            duplicates[k] = v
    return pd.DataFrame.from_dict(duplicates, orient='index', columns=['# of Duplicates'])

def defining_df(df, column_range, number_col):
    cont = 0
    for i in df.iloc[:,number_col]:
        if pd.isnull(i):
            break # acertar o break
        else:
            cont +=1
    df = df.iloc[0:cont, :]
    return df

#old version
def check_columns(table, output_columns):
    """
    Check the total of number of missing columns and the missing columns in passed table.\n

    Params:\n
    table: contain the columns to be check.\n
    output_columns: columns structure at the final file. 

    Returns:\n
    Number of missing columns and a list that contains the name os missing columns.
    """
    """"
    countries = ['DE':{'towerDB':[],\
                       'lc': [],\
                       'ta':[]}\,
                 'HU',
                 'IE',
                 'RO',
                 'PT',
                 'ES',
                 'CZ': {'towerDB':["Code (Duplicate)","Site Status","VF - In scope / out of scope (Generalised scoping)","Site in Skylon scope Actual (From Site List Sheet )","Legacy Site Code(Duplicate)","TIMS Site Code","Legacy Site Code","Site Name","Macro Region","Province","Municipality","Inhabitants","Address","Ground Register","Altitude","Latitude","Longitude","Categorization by Inhabitants","Categorization by Transmission Sys","Categorization by Site Type","Categorization by Transmission Sys (subcluster)","Other internal Categorization 1 (Identify ACQ Sites)","Other internal Categorization 2 Energy provider (Eon/ LL)","DAS+Macro","DAS (Yes/ No)","DAS Ownership (Complete/ Partial/ 3rd Party)","Active/ Passive DAS","# of remote units/ radiating points","Type of Structure","Distance highest antenna to ground level","GBT Tower height","POD ID","Energy Consumption LTM (kwh)","Annual Energy cost LTM (Euros)","Infrastructure ready (existing)/ to be ready (new)","Infrastructure to be dismantled by","Radio equipments to be deactivated by","Infrastructure to be shared by","Technology VOD","Fibre / Microwave","Vertical passive structure owner","Room configuration (detailed)","Shelter passive structure ownership","Type of Air Conditioning","Number of cabinets (Full Capacity)","Number of Antenna (Full Capacity)","Number of MW (Full Capacity)","Counterpart","# of Lease Contracts","Current annual lease fees ","Current other fees (Maintenance)","Current other fees","(Average) residual duration - Lease contract","(Average) residual duration - Maintenance contract","(Average) residual duration - Lease & Maintenance both","# of Tenants Agreements","Current Total Annual Hosting Fees","Tenant (name/ID) MNO1 (Česká telekomunikační infrastruktura a.s.)","Annual Fee per Tenant MNO1","Annual Energy Fee MNO1","Annual Maintenance Fee MNO1","Other Services Fee MNO1","Residual duration MNO1 (Years)","Tenant (name/ID) MNO2 (T-Mobile Czech Republic a.s.)","Annual Fee per Tenant MNO2","Annual Energy Fee MNO2","Annual Maintenance Fee MNO2","Other Services Fee MNO2","Residual duration MNO2 (days)","Tenant (name/ID) MNO3","Annual Fee per Tenant MNO3","Annual Energy Fee MNO3","Annual Maintenance Fee MNO3","Other Services Fee MNO3","Residual duration MNO3","# of OTMOs","Annual Fee from OTMOs","Annual Energy Fee from OTMOs","Annual Maintenance Fee OTMOs","Other Services Fee OTMOs","Average residual duration (days)","Check","Strategic Macro Sites","Critical Sites","Case A Core Site","Macro Site - Transmission Hub Site","Macro Site - Transmission Hub Site with/without Shelters","Transmission Sites","Room Configuration","Power Supply","Air Conditioning","Active Sharing Arrangements involving the Operator","VF-CZ Demerger phase","EVO Location [FAR Site ID] ","Billing Trigger date ",\
                 "Strategic_Site_Bucket","Critical Site Bucket","Subsequent_Sharing_Arrangement","First_Active_Sharing_Deployment_Type","First_Active_Sharing_Start_Date","First_Active_Sharing_End_Date","Decommissioned_Site","Wip_Site","Bts_Site","Transfer_Date_Of_Consent_Required_Sites","Sites_As_Metered_Estimated","Date_Of_Equipment_Removal"],\
                       'lc': [],\
                       'ta':['Tenant Agreement ID - NEW', 'Code', 'Site Name', 'Service Type','Contract start date', 'Contract end date', 'Status of contract','Renewal Option', 'Comments', 'Counterpart ID', 'Counterpart','Classification of Tenant', 'Annual amount - billing currency','Billing Currency', 'Annual Amount in CZK', 'Amount in EUR','Terms of Payment', 'Frequency', 'Indexed YES/NO', 'Index','Percentage', 'VAT Subject YES/NO', 'Percentage (VAT)', 'Unique Key','Classification', 'Residual Period in Years', 'Remarks','Count of unique key', 'Count of contracts', 'Scoping classification','DAS sites', 'DAS ownership', '31-Mar-21', 'Update in month','Updated Item', 'Comment', 'Counterpart_extra_1', 'Counterpart_extra_2', 'x','SiteType', '1.028', 'Unique Tenant', 'Unique Tenant Count', 'not_def_1']}\,
                 }
                 'GR']
                 """
    total_received = len(table.columns)
    number_missing_columns = 0
    missing_columns = []
    #Counting of missing columns       
    #if country in contries:      
    for columns in output_columns:
        if columns.lower() not in [labels.lower() for labels in table]:
            number_missing_columns +=1
            missing_columns.append(columns)
    
    return total_received, number_missing_columns, missing_columns

def check_columns_received(df, bill_cols):
    twdb_col = lower_str(list(df.columns))
    col_miss = [i for i in bill_cols if i not in twdb_col]

    df_col_missing = pd.DataFrame(col_miss, columns=['Column(s) Missing'], index=range(len(col_miss)))
    return df_col_missing

def replace_values(df, columns, value=""):
    """
    Está voltando para float
    """
    invalid_values = ['N/A', 'n/a',"0", 0, '-', '_', np.nan,'nan']
    #lista = []

    df.fillna(0, inplace=True)
    #df[column] = df[column].astype('int64')

    for column in columns:
        lista = []
        for index in df[column]:
            #print(f"{column} -> {index}")
            if index in invalid_values:
                lista.append(value)
            else:
                lista.append(index)
        df[column] = lista

    return df
      
def date_parser(df, columns, format, type_dates):
    t_col = type_dates.lower()
    for column in columns:
        if t_col == 'mixed':
            df[column] = [date_obj.strftime(format) if not pd.isnull(date_obj) \
                        and not isinstance(date_obj, str) else date_obj for date_obj in df[column]]
            df[column] = df[column].astype(str)
        else:
            df[column] = [date_obj.strftime(format) if not pd.isnull(date_obj) else '' for date_obj in df[column]]
            df[column] = df[column].astype(str)

# Refactorar esse codigo para receber todas as colunas num dic
# Sendo as keys=columns e values= picklist for each column
def check_date_columns(df, df_index, columns, format):
    """
    Paramns \n
        Dates needs to be in string format not in datetime, otherwise raise an error.\n
        Convert entire column to string before.
    """
    df_dates = df[columns]
    df_dates['sites'] = df_dates[df_index]
    df_dates = df_dates.set_index('sites')

    df_de = pd.DataFrame(columns=columns)
    
    df_duplicates = count_duplicates(df_dates[df_index])
    if not df_duplicates.empty:
        df_dates.drop_duplicates(subset=[df_index], inplace=True)
    
    if format == 1:
        date_format = re.compile(r'\d{2}\-\d{2}\-\d{4}')
        for de_site in df_dates[df_index]:
            for de_column in columns[1:]:
                #match = re.search(r'\d{2}\/\d{2}\/\d{4}', df_dates.loc[ta_site,ta_column])
                #print(type(df_dates.loc[de_site,de_column]))
                if date_format.match(df_dates.loc[de_site,de_column]) == None:
                    if df_dates.loc[de_site,df_index] not in df_de.index:
                        df_de.loc[de_site,df_index] = df_dates.loc[de_site,df_index]
                        df_de.loc[de_site,de_column] = 'Incorret Format or Blank Value'
                    else:
                        df_de.loc[de_site,de_column] = 'Incorret Format or Blank Value'
        df_de = df_de.dropna(how='all', axis=1).fillna('Ok')       
        if not df_de.empty:
            return df_de
        else: 
            print('No one columns with incorrect date format!')
    else:
        date_format = re.compile(r'\d{2}\/\d{2}\/\d{4}')
        for de_site in df_dates[df_index]:
            for de_column in columns[1:]:
                #match = re.search(r'\d{2}\/\d{2}\/\d{4}', df_dates.loc[ta_site,ta_column])
                if date_format.match(df_dates.loc[de_site,de_column]) == None:
                    if df_dates.loc[de_site,df_index] not in df_de.index:
                        df_de.loc[de_site,df_index] = df_dates.loc[de_site,df_index]
                        df_de.loc[de_site,de_column] = 'Incorret Format or Blank Value'
                    else:
                        df_de.loc[de_site,de_column] = 'Incorret Format or Blank Value'
        df_de = df_de.dropna(how='all', axis=1).fillna('Ok')       
        if not df_de.empty:
            return df_de
        else: 
            print('No one columns with incorrect date format!')

def check_amounts(df_check, df_index, columns, pattern=','):
    """
    Paramns:
    pattern: general (. to decimal), other (, to decimal)
    """

    df = df_check[columns]
    df['sites'] = df[df_index]
    df = df.set_index('sites')

    df_new = pd.DataFrame(columns=columns)
    
    df_duplicates = count_duplicates(df[df_index])
    if not df_duplicates.empty:
        df.drop_duplicates(subset=[df_index], inplace=True)

    for site in df[df_index]:
        for column in columns[1:]:
            if not str(df.loc[site,column]).__contains__(pattern):
                #print(df.loc[site,column])
                if df.loc[site,df_index] not in df_new.index:
                    df_new.loc[site,df_index] = df.loc[site,df_index]
                    df_new.loc[site,column] = 'Incorret Format to separate decimal values'
                else:
                    df_new.loc[site,column] = 'Incorret Format to separate decimal values'

    df_new = df_new.dropna(how='all', axis=1).fillna('Ok')       
    if not df_new.empty:
        return df_new
    else: 
        print('No one columns with incorrect Amount format!')
        
def check_picklist(df,df_index, df_status, df_cols, picklist_dict):
    """
    Params:
        df:
        df_index: 
        picklist_dict:
        nultiIndex: More than one column to check
    """
    df_picklist = df[df_cols]
    df_picklist['sites'] = df[df_index]
    df_picklist =  df_picklist.set_index('sites')
    
    #df_picklist = replace_values(df_picklist, df_cols, 0)

    df_errors = pd.DataFrame(columns=df_cols)

    for site in df[df_index]:
        columns = [i for i in picklist_dict.keys()]
        for column in set(columns): 
            value = str(df_picklist.loc[site,column])
            #print(value)
            if not value in set(picklist_dict[column]) or pd.isnull(value):
                #print(set(picklist_dict[column]))

                if not df_picklist.loc[site,df_index] in df_errors.index:
                    df_errors.loc[site,df_index] = df_picklist.loc[site,df_index]
                    
                    if pd.isnull(value) or pd.isna(value) or value == 'nan' or value == '':
                        df_errors.loc[site,column] = 'Blank Value'
                    else:
                        df_errors.loc[site,column] = f'Incorret picklist value: {value}'
                else:
                    
                    if pd.isnull(value) or pd.isna(value) or value == 'nan' or value == '':
                        df_errors.loc[site,column] = 'Blank Value'
                    else:
                        df_errors.loc[site,column] = f'Incorret picklist value: {value}'

    #df = df_errors.dropna()   
    df_errors = df_errors.dropna(how='all', axis=1).fillna('Ok!')
    if len(df_errors)>0:
        df = df[[df_index, df_status]]
        df_errors = pd.merge(df_errors,df, how='left', on=[df_index])
        df_errors = df_errors.set_index(df_index)
        df_errors = df_errors[[df_status]+ df_errors.columns[:-1].tolist()]
        df_errors = df_errors.reset_index()
    else:
        print('\nNo one Picklist Error Founded!')
    return df_errors

def check_picklist_3(df,df_index, picklist_dict):
    log = {}
    for column in picklist_dict:
        df_aux = df.copy()
        new = df_aux[column].isin(picklist_dict[column])
        #new = df_aux[column].apply(lambda x: x in picklist_dict[column])
        # Aceita somento os valores que não estão na picklist
        indexes = df.index[new == False].tolist()
        #if len(indexes)>0:
        if column not in log:log[column]=[]
        log[column]=log[column]+indexes
    #print(log.keys())

    newDict ={}
    df1 = pd.DataFrame()
    for key,value in log.items():
        for val in value:  
            ID=df.iloc[val][df_index]
            if ID in newDict:
                newDict[ID].append(key)
            else:
                newDict[ID] = [key]
        
    logs = pd.DataFrame.from_dict(newDict, orient='index')
    return logs

#check on air foi trocado pelo check_picklist
def check_on_air_sites(df, df_index, df_cols):

    df_check = df[df_cols]
    df_check['sites'] = df_check[df_index]
    df_check = df_check.set_index('sites')
    #df_check = replace_values(df_check, df_cols, 0)

    df_errors = pd.DataFrame(columns=df_cols)

    for site in df[df_index]:
        for column in df_cols: 
            if df_check.loc[site,df_index] not in df_errors.index:
                df_errors.loc[site,df_index] = df_check.loc[site,df_index]
                if df_check.loc[site,column] == 0:
                    df_errors.loc[site,column] = 'Incorret picklist value or Blank Value'
            else:
                if df_check.loc[site,column] == 0:
                    df_errors.loc[site,column] = 'Incorret picklist value or Blank Value'
    df = df_errors[df_errors.iloc[:,1:]]
    df = df.dropna(how='all', axis=1).fillna('Ok')       
    #df = df_errors.dropna(how='all', axis=1)   
    return df

def check_new_sites(df_towerdb, tw_index, bts_col, bill_col, msa_list, towerdb_list, uip_list):
    
    # capture current date as string
    current_date = pd.to_datetime('now').date()
    # convert current to timestamp
    current_date = pd.to_datetime(current_date)
    
    #Finding for ALL NEW SITES
    new_site = [str(i) for i in towerdb_list if i not in msa_list]
    new_sites = pd.DataFrame(new_site, columns=['New_Sites'])
    
    #list of new sites out of UIP sheet
    bts_out_uip = [str(i) for i in df_towerdb[df_towerdb[bts_col]=='Yes'][tw_index].sort_values() if i not in uip_list]
    bts_out_uip = pd.DataFrame(bts_out_uip, columns=['Bts_Sites_Out_UIP_File'])

    #create a copy to make index modifications
    df = df_towerdb.copy()

    #New columns with Site Codes
    df['Sites'] = df[tw_index]
    #defining site code to index
    df.set_index('Sites', inplace=True)

    #filtering by new sites
    df = df[df[tw_index].isin(new_site)]

    # Save information os sites with demerged date more than current date
    df[bill_col] = df[bill_col].astype('datetime64[s]')
    df_site_bts = df[(df[bts_col]=='Yes') | (df[bill_col] > current_date)]

    return new_sites, bts_out_uip, df_site_bts[[tw_index, bts_col, bill_col]]
    """Restruturar o script em CZ, DE, PT"""

def check_bts(df_tw, bts_tw_columns, tw_index, df_msa, bts_msa_column, msa_index):
    #Nested Function to make conditional validations
    def cond_bts_check(bts_msa, tw_bts_sites):
        bts_out_tw=[]
        if sorted(bts_msa) != sorted(tw_bts_sites):
            for i in tw_bts_sites:
                if i not in bts_msa:
                    bts_out_tw.append(i)

        return bts_out_tw

    bts_msa = msa[msa[bts_msa_column]=='Yes']
    bts_msa = [str(i) for i in bts_msa[msa_index]]

    tw_bts_sites = df_tw[df_tw[bts_tw_columns]=='Yes']
    tw_bts_sites = [str(i) for i in tw_bts_sites[tw_index]]

    #return of datas
    bts_out_tw = cond_bts_check(bts_msa, tw_bts_sites)
    df = pd.DataFrame(bts_out_tw, columns=['New Sites'])
    return df

def check_wip(df_tw,tw_index, wip_tw, tw_bts, df_msa, msa_index, wip_msa_col):

    #Nested Function to make conditional validations
    def cond_wip_check(wip_msa, tw_wip_sites):
        count_wip = 0
        wip_out_tw=[]
        if sorted(wip_msa) != sorted(tw_wip_sites):
            for i in tw_wip_sites:
                if i not in wip_msa:
                    count_wip += 1
                    wip_out_tw.append(i)

        return wip_out_tw

    wip_msa = df_msa[df_msa[wip_msa_col]=='Yes']
    wip_msa = [str(i) for i in wip_msa[msa_index]]

    tw_wip_sites = df_tw[df_tw[wip_tw]=='Yes']
    tw_wip_sites = [str(i) for i in tw_wip_sites[tw_index]]

    tw_wip_site_bts_flagged = df_tw[(df_tw[wip_tw]=='Yes')&(df_tw[tw_bts]=='Yes')]
    tw_wip_site_bts_flagged = tw_wip_site_bts_flagged[[tw_index, wip_tw, tw_bts]]

    wip_out_tw_list = cond_wip_check(wip_msa, tw_wip_sites)
    return wip_out_tw_list, tw_wip_site_bts_flagged
    """Reestrurar os script em PT, DE, CZ"""
    # Falta os outros países

def check_decommissioned(df,df_index, decom_col, doer_col):
    #c = country.lower()
    filtered = df[(df[decom_col]=='Yes')&(df[doer_col]=="")]
    return filtered[[df_index, decom_col, doer_col]]
    """Ajustar para CZ, DE, PT"""
    
def check_tw_bill_doer(df_tw, tw_index, date_col, status_col, status, type_col):
    
    t = type_col.lower()
    # capture current date as string
    current_date = pd.to_datetime('now').date()
    # convert current to timestamp
    current_date = pd.to_datetime(current_date)
    if not df_tw[date_col].empty:
        if t == 'doer':
            filtered = df_tw[(df_tw[status_col]==status)&(df_tw[date_col].astype('datetime64[ns]') < current_date)]
            return filtered[[tw_index, status_col, date_col]] 
        else:
            filtered = df_tw[(df_tw[status_col]==status)&(df_tw[date_col].astype('datetime64[ns]').empty)]
            return filtered[[tw_index, status_col, date_col]] 
    else:
        print('Nothing wrong in services sites!')


def check_tw_doer_planned(df_tw, tw_index, doer_col, status_col):
    """Only GR until now"""
    # capture current date as string
    current_date = pd.to_datetime('now').date()
    # convert current to timestamp
    current_date = pd.to_datetime(current_date)
    if not df_tw[doer_col].empty:
        filtered = df_tw[(df_tw[status_col]=='Planned')&(not df_tw[doer_col].astype('datetime64[ns]').empty)]
        return filtered[[tw_index, status_col, doer_col]]  
    else:
        print('Nothing wrong in services sites!')
                                                              
def check_mom_bts(df_tw, tw_index, tw_col, df_msa,msa_index, msa_col):

    #c = country   
    msa_bts = df_msa[df_msa[msa_col]=='Yes']
    msa_bts_sites = [i for i in msa_bts[msa_index]]

    tw_bts = df_tw[df_tw[tw_col]=='Yes']
    tw_bts_sites = [i for i in tw_bts[tw_index]]

    out_tower_bts = [i for i in msa_bts_sites if i not in tw_bts_sites]
    filtered = tw_bts[tw_bts[tw_index].isin(out_tower_bts)]
    return filtered[[tw_index, tw_col]]         

def check_lc_ta_dates(df,tw_index, start_date,end_date):

        filtered = df[df[start_date] > df[end_date]]
        return filtered[[tw_index, start_date,end_date]]

def check_uip_tw(df_tw,tw_index, status_tw_col, decom_col, tw_bts_col, tw_critical_col, df_uip, uip_sites, country):
    t1 = ['pt', 'de', 'cz', 'ie', 'es', 'ro', 'hu']
    if country.lower() in t1:
        #tw_active = df_tw[df_tw['Site Status']=='In Service']
        count_tw_sites = [i for i in df_tw[df_tw[status_tw_col] =='In Service'][tw_index]]

        # check number of sites that are in uip file and doesn't have in df_tw
        in_service_uip_sites = []
        if not set(count_tw_sites).intersection(uip_sites):
            in_service_uip_sites = [i for i in uip_sites if i not in count_tw_sites]
        in_service_uip_sites = pd.DataFrame(in_service_uip_sites,columns=['Site In service out of UIP File!'])
        
        #check for decomissioned site not in uip files
        if decom_col != "":
            tw_decomiss = [i for i in df_tw[df_tw[decom_col]=='Yes'][tw_index] if i in uip_sites]
            decomiss_sites_in_uip = pd.DataFrame(tw_decomiss, columns=['Decomissioned Site in UIP File'])
        
            #Check BTS sites
            bts_sites = [i for i in df_tw[df_tw[tw_bts_col]=='Yes'][tw_index]]
            bts_sites_out_uip = []
            if not set(bts_sites).intersection(uip_sites):
                bts_sites_out_uip = [i for i in bts_sites if i not in uip_sites]
            bts_sites_out_uip = pd.DataFrame(bts_sites_out_uip, columns=['BTS Site not in UIP File'])

            #  Check for UIP critical sites 
            uip_critical = [i for i in df_uip[(df_uip['Commercials for sites beyond 10% cap of critical sites (Annual)']!='')&\
                                        df_uip['BTS site applicable charge (Annual)']!=""]['Site_ID']]
            bts_tw_critical = df_tw[df_tw[tw_critical_col]=='Beyond 10%'][tw_index]
            critical = []
            if len(uip_critical) > 0:
                if set(uip_critical).intersection(bts_tw_critical):
                    critical = [i for i in bts_tw_critical if i not in uip_critical]
            critical = pd.DataFrame(critical, columns=['Sites with critical level beyond 10% in out UIP File'])

        
            return in_service_uip_sites, decomiss_sites_in_uip, bts_sites_out_uip, critical
        else:
            #Check BTS sites
            bts_sites = [i for i in df_tw[df_tw[tw_bts_col]=='Yes'][tw_index]]
            bts_sites_out_uip = []
            if not set(bts_sites).intersection(uip_sites):
                bts_sites_out_uip = [i for i in bts_sites if i not in uip_sites]
            bts_sites_out_uip = pd.DataFrame(bts_sites_out_uip, columns=['BTS Site not in UIP File'])

            #  Check for UIP critical sites 
            uip_critical = [i for i in df_uip[(df_uip['Commercials for sites beyond 10% cap of critical sites (Annual)']!='')&\
                                        df_uip['BTS site applicable charge (Annual)']!=""]['Site_ID']]
            bts_tw_critical = df_tw[df_tw[tw_critical_col]=='Beyond 10%'][tw_index]
            critical = []
            if len(uip_critical) > 0:
                if set(uip_critical).intersection(bts_tw_critical):
                    critical = [i for i in bts_tw_critical if i not in uip_critical]
            critical = pd.DataFrame(critical, columns=['Sites with critical level beyond 10% in out UIP File'])
            return in_service_uip_sites, bts_sites_out_uip, critical
    else:
        #tw_active = df_tw[df_tw['Site Status']=='In Service']
        count_tw_sites = [i for i in df_tw[df_tw[status_tw_col] =='In Service'][tw_index]]

        # check number of sites that are in uip file and doesn't have in df_tw
        in_service_uip_sites = []
        if not set(count_tw_sites).intersection(uip_sites):
            in_service_uip_sites = [i for i in uip_sites if i not in count_tw_sites]
        in_service_uip_sites = pd.DataFrame(in_service_uip_sites,columns=['Site In service out of UIP File!'])
        
        #check for decomissioned site not in uip files
        tw_decomiss = [i for i in df_tw[df_tw[decom_col]=='Yes'][tw_index]]
        decomiss_sites_in_uip = []
        if set(tw_decomiss).intersection(uip_sites):
            decomiss_sites_in_uip = [i for i in tw_decomiss if i in uip_sites]
        decomiss_sites_in_uip = pd.DataFrame(decomiss_sites_in_uip, columns=['Decomissioned Site in UIP File'])
        
        #Check BTS sites
        bts_sites = [i for i in df_tw[df_tw[tw_bts_col]=='Yes'][tw_index]]
        bts_sites_out_uip = []
        if not set(bts_sites).intersection(uip_sites):
            bts_sites_out_uip = [i for i in bts_sites if i not in uip_sites]
        bts_sites_out_uip = pd.DataFrame(bts_sites_out_uip, columns=['BTS Site not in UIP File'])

        #  Check for UIP critical sites 
        uip = [i for i in df_uip['Site_ID']]
        bts_tw_critical = df_tw[df_tw[tw_critical_col]=='Yes'][tw_index]
        critical = []
        if set(uip).intersection(bts_tw_critical):
            critical = [i for i in bts_tw_critical if i not in uip]
        critical = pd.DataFrame(critical, columns=['Sites with critical level beyond 10% out in UIP File'])

        return in_service_uip_sites, decomiss_sites_in_uip, bts_sites_out_uip, critical

def check_commercial(path_current, path_before, col_replace, col_names, merge_cols, col_order):
    
    def check_uip_commercial_values(df_actual, df_before, merge_cols):
        df_atual = uip_comercial_actual
        df_ant = uip_comercial_before
        df_cross = pd.merge(df_actual, df_before, on=merge_cols, \
                            how='left', suffixes=('_actual', '_before'), indicator='Exist')
        df_cross['Exist'] = np.where(df_cross.Exist == 'both', 'Yes', 'No')
        df_cross['Equal Values'] = df_cross['Exist']
        
        return df_cross
    # Check for commercial Values into current UIP File and compare with UIP File before
    uip_comercial_actual = pd.read_excel(path_current,sheet_name='Commercial', names=col_names)
    #uip_comercial_actual = uip_comercial_actual[['Charge_Type', 'Data_Type', 'Input_Value']]
    uip_comercial_actual = replace_values(uip_comercial_actual, col_replace, value=0)

    uip_comercial_before = pd.read_excel(path_before,sheet_name='Commercial', names=col_names )
    #uip_comercial_before = uip_comercial_before[['Charge_Type', 'Data_Type', 'Input_Value']]
    uip_comercial_before = replace_values(uip_comercial_before, col_replace, value=0)

    df_commercial =  check_uip_commercial_values(uip_comercial_actual, uip_comercial_before, merge_cols)

    df_commercial = df_commercial.reindex(columns=col_order)
    df_commercial_diffs = df_commercial[df_commercial['Equal Values']=='No']
    return df_commercial_diffs

def general_log_erros(df_list, sheet_list, path):
    writer = pd.ExcelWriter(path,engine='openpyxl')   
    for dataframe, sheet in zip(df_list, sheet_list):
        dataframe.to_excel(writer, sheet_name=sheet, startrow=0 , startcol=0)   
    writer.save() 

def find_diffs_between_files(path_OLD, path_NEW, index_col, bill_cols, \
                             status_col, path_save, type_file='mix',kind='tw', sheetname='', skipr=0, skipc=0):
    
    def lower_str(columns):
        newlist = list(map(lambda x: x.lower(), columns))
        return newlist

    def fit_df(df, bill_cols):
        fit_cols = lower_str(list(df.columns))
        df.columns = fit_cols
        df = df[bill_cols]
        df = df.dropna(subset=[index_col], axis=0)
        df = df.drop_duplicates(subset=[index_col], keep='last')
        df[index_col] = df[index_col].astype(str)
        df['sites'] = df[index_col]
        # Aqui ajusta os nan e nat dos DFs, quando não forem arquivos não lidos
        df = df.set_index('sites').fillna('')
        return df
        
    def change_format(index, worksheet, format):
        for cell in worksheet[f"{str(index)}:{str(index)}"]:
            cell.font = format

    bill_cols = lower_str(bill_cols)
    index_col = index_col.lower()
    type_file = type_file.lower()
    status_col = status_col.lower()
    if type_file == 'excel':
        df_OLD = pd.read_excel(path_OLD,sheet_name = sheetname, skiprows = skipr).fillna('')
        df_OLD = df_OLD.iloc[:,skipc:]
        df_OLD = fit_df(df_OLD,bill_cols)

        df_NEW = pd.read_excel(path_NEW,sheet_name = sheetname, skiprows = skipr).fillna('')
        df_NEW = df_NEW.iloc[:,skipc:]
        df_NEW = fit_df(df_NEW,bill_cols)

    elif type_file == 'csv':
        df_OLD = pd.read_csv(path_OLD, encoding='latin').fillna('')
        df_OLD = fit_df(df_OLD,bill_cols)

        df_NEW = pd.read_csv(path_NEW, encoding='latin').fillna('')
        df_NEW = fit_df(df_NEW,bill_cols)

    elif type_file == 'mix':
        df_OLD = pd.read_csv(path_OLD, encoding='latin')
        df_OLD = fit_df(df_OLD,bill_cols)

        df_NEW = pd.read_excel(path_NEW,sheet_name = sheetname, skiprows = skipr)
        df_NEW = df_NEW.iloc[:,skipc:]
        df_NEW = fit_df(df_NEW,bill_cols)
    else: 
        df_OLD = fit_df(path_OLD,bill_cols)
        df_NEW = fit_df(path_NEW,bill_cols)


    # Perform Diff
    new_copy = df_NEW.copy()
    droppedRows = []
    newRows = []

    cols_OLD = df_OLD.columns
    cols_NEW = df_NEW.columns
    sharedCols = list(set(cols_OLD).intersection(cols_NEW))

    for row in new_copy.index:
        if (row in df_OLD.index) and (row in df_NEW.index):
            for col in sharedCols:
                value_OLD = df_OLD.loc[row,col]
                value_NEW = df_NEW.loc[row,col]
               #Error de a.empty() são sites duplcados e nã poodem estar no index
                if value_OLD == value_NEW:
                    new_copy.loc[row,col] = np.nan
                else:
                    new_copy.loc[row,col] = f'{value_OLD} > {value_NEW}'
        else:
            newRows.append(row)

    new_copy = new_copy.dropna(axis=0, how='all')
    new_copy = new_copy.dropna(axis=1, how='all')

    for row in df_OLD.index:
        if row not in df_NEW.index:
            droppedRows.append(row)
            new_copy = new_copy.append(df_OLD.loc[row,:])
    
    new_copy = new_copy.sort_index(key=lambda x: x.str.lower()).fillna('')
    
    new_copy = new_copy.reset_index()
    if kind=='tw':
        sites = [i for i in new_copy['sites']] 
        old = df_OLD[[status_col]].reset_index()
        old = old.loc[old['sites'].isin(sites)]
        new = df_NEW[[status_col]].reset_index()
        new = new.loc[new['sites'].isin(sites)]
        df_cross = pd.merge(new, old, on=['sites'], how='inner', suffixes=('_current', '_before'))
        new_copy = pd.merge(new_copy, df_cross, on=['sites'], how='left')


    print(f'\nNew Rows:     {newRows}')
    print(f'Dropped Rows: {droppedRows}')

    # Save output and format
    fname = f'{path_save}_old_file vs new_file.xlsx'
    file = pd.ExcelWriter(fname, engine='openpyxl')
    
    new_copy.to_excel(file, sheet_name='diffs_founded', index=False)
    df_NEW.to_excel(file, sheet_name='new_file', index=False)
    df_OLD.to_excel(file, sheet_name='old_file', index=False)

    # get openpyxl objects
    wb  = file.book
    ws = file.sheets['diffs_founded']
    
    red_fill = PatternFill(start_color='95A7B3', \
                                end_color='95A7B3', fill_type='solid')
    red_font = Font(color='00FF0000', italic=True)
    new_fmt = Font(color='3F976D',bold=True, italic=True)
    removed = Font(color='95A7B3',bold=True, italic=True)

    dxf = DifferentialStyle(font=red_font, fill=red_fill)
    highlight = Rule(type="containsText", operator="containsText", text=">", dxf=dxf)
    highlight.formula = ['SEARCH(">", A1)']
    ws.conditional_formatting.add('A1:ZZ1000', highlight)
    
     
    #print(len(newRows))
    for site in new_copy['sites']:
        if site in newRows:
            idx = new_copy.index[new_copy[index_col]==site].to_list()
            idx = list(map(lambda x: x+2, idx))
            change_format(*idx,ws,new_fmt)
        if site in droppedRows:
            idx = new_copy.index[new_copy[index_col]==site].to_list()
            idx = list(map(lambda x: x+2, idx))
            change_format(*idx,ws,removed)
    
    wb.save(fname)
    print('\nDone.\n')


In [ ]:
tw_bills = ['Site ID ',\
            'Strategic',\
            'Critical',\
            'Site Type (See structure Type)',\
            'Core Site Type',\
            'Transmission System',\
            'Macro Site - Transmission Hub Site',\
            'Macro Site - Transmission Hub Site with/without Shelters',\
            'Transmission Sites',\
            'Room Configuration',\
            'Power Supply',\
            'Air Conditioning',\
            'Site Status',\
            'Strategic_Site_Bucket',\
            'CriticalSite_Beyond_10',\
            'Sites_As_Metered_Estimated',\
            'Meter_Sharing_Site',\
            'Bts_Sites',\
            'Phase1_Site_Or_Consent_Site',\
            'Transfer_Date_Of_Consent_Site',\
            'WIP_Sites',\
            'Active_Sharing_Arrangement',\
            'Subsequent_Sharing_Arrangement',\
            'First_Active_Sharing_Deployment_Type',\
            'First_Active_Sharing_Start_Date',\
            'First_Active_Sharing_End_Date']
pathtw='/content/TowerDB_Ireland_20210531 1.xlsx'
pathold = '/content/TowerDB_Ireland_20210630.csv'
sheet='TowerDB_Input_Ireland_20210531'
skiprows=0
skipcols=0
to_save = '/content/IE_TW'
find_diffs_between_files(pathold, pathtw, 'Site ID ', tw_bills,'Site Status', to_save,'mix', sheetname=sheet)

Adding new columns for UIP



In [ ]:
# Lendo o Ficheiro de input
path_uip=''
sheet = 'SiteLevel'
site_level = pd.read_excel(path_uip,sheet, header=[0,1,2])
head_1 = pd.MultiIndex.from_product([[''],['Numeric (in months)'],['Delay in Site Modification Projects', 'Delay in BTS Projects']])
df_1 = pd.DataFrame(columns=head_1)

head_2 = pd.MultiIndex.from_product([[''],['Numeric (in €)'],['Excess of Upgrade Capital Expenditure over Threshold']])
df_2 = pd.DataFrame(columns=head_2)

site_level = pd.concat([site_level, df_1, df_2], axis=1)
site_level.head(1)

Towerdb File

In [ ]:
def read_files(path, sheetname, n_skiprows, n_skip_columns, site_index, col_dates, bill_col, format='mix', type_date="%d/%m/%Y"):
    def lower_str(columns):
        newlist = list(map(lambda x: x.lower(), columns))
        return newlist

    def replace_values(df, columns, value=""):
        """
        Está voltando para float
        """
        invalid_values = ['N/A', 'n/a',"0",0, '-', '_', np.nan,'nan', ' ']
        #lista = []

        #df[column] = df[column].astype('int64')

        for column in columns:
            lista = []
            df[column] = df[column].fillna(0)
            for index in df[column]:
                #print(f"{column} -> {index}")
                if index in invalid_values:
                    lista.append(value)
                else:
                    lista.append(index)
            df[column] = lista

        return df

    def date_parser(df, columns,format, type_date):
        for column in columns:
            if format == 'mix':
                #df[column] = df[column].fillna('')
                df[column] = [date_obj.strftime(type_date) if not pd.isnull(date_obj) \
                            and not isinstance(date_obj, str) else date_obj for date_obj in df[column]]
                df[column] = df[column].astype(str)
            else:
                #df[column] = df[column].fillna('')
                df[column] = [date_obj.strftime(type_date) if not pd.isnull(date_obj) else '' for date_obj in df[column]]
                df[column] = df[column].astype(str)

    df = pd.read_excel(path, sheet_name = sheetname, engine='openpyxl', skiprows = n_skiprows).fillna('')
    df = df.iloc[:,n_skip_columns:]
    #df = df.rename(columns={'Billing Trigger Date': 'Billing Trigger Date2'})
    df = replace_values(df,col_dates)
    date_parser(df, col_dates, format, type_date)
    # define the entiry columns which doesn't have NaN 
    df = df.dropna(subset=[site_index], axis=0)
    df = df.replace(['N/A', 'n/a',"0", '-', '_', np.nan,'nan', ' '], '')

    """for col in cols_number:
        df[col] = df[col].astype(int).apply(lambda x: f'{x:,}')"""
    
    df = df.fillna('')

    return df
    
""" Need Unlock spreadsheet before"""
tw_cols = ["Site ID ","Population","County","Site Name ","Address ","Site Provider","Strategic","Critical",\
           "Altitude (ASL)","Lat (WGS 84)","Long (WGS 84)","Site Class ","Site Owner","Hub List ",\
           "Site Type (See structure Type)","Active Dishes","INTP ","Core Site Type","Transmission System",\
           "Macro Site - Transmission Hub Site","Macro Site - Transmission Hub Site with/without Shelters",\
           "Transmission Sites","Public DAS Sites (Active/ Passive)","Fibre/Microwave ","Technology VOD",\
           "Structure Type ","Structure Type (Standardised)","Room Configuration","Power Supply","Air Conditioning",\
           "Tower Height ","Site Area (sqm)","Site Area (Available) ","Macro Region","MPRN","Site Decom",\
           "SAP Site Code","Energy Cost FY1819","Active / Not Active","New Build planned ","Site Status",\
           "Planned Sublets ","Infrastructure Ready Date","Strategic_Site_Bucket","CriticalSite_Beyond_10",\
           "Sites_As_Metered_Estimated","Meter_Sharing_Site","Bts_Sites","Phase1_Site_Or_Consent_Site",\
           "Transfer_Date_Of_Consent_Site","Billing Trigger Date","WIP_Sites","Decommisioned_Site",\
           "Active_Sharing_Arrangement","Subsequent_Sharing_Arrangement","First_Active_Sharing_Deployment_Type",\
           "First_Active_Sharing_Start_Date","First_Active_Sharing_End_Date","BTS Site Acceptance Date",\
           "Billing Trigger Date2","Date of Equipment Removal"]

bill_cols = ['Site ID ','Population','County','Site Name ','Address ','Site Provider','Strategic','Critical','Altitude (ASL)','Lat (WGS 84)','Long (WGS 84)',\
             'Site Class ','Site Owner','Hub List ','Site Type (See structure Type)','Active Dishes','INTP ','Core Site Type','Transmission System',\
             'Macro Site - Transmission Hub Site','Macro Site - Transmission Hub Site with/without Shelters','Transmission Sites','Public DAS Sites (Active/ Passive)',\
             'Fibre/Microwave ','Technology VOD','Structure Type ','Structure Type (Standardised)','Room Configuration','Power Supply','Air Conditioning',\
             'Tower Height ','Site Area (sqm)','Site Area (Available) ','Macro Region','MPRN','Energy Consumption FY1819 (KWh)','Energy Consumption FYTD (KWh)',\
             'Energy Cost FY1819','Energy Cost FYTD','New Build planned ','Site Status','Planned Sublets ','Infrastructure Ready Date','Strategic_Site_Bucket',\
             'CriticalSite_Beyond_10','Sites_As_Metered_Estimated','Meter_Sharing_Site','Bts_Sites','Phase1_Site_Or_Consent_Site','Transfer_Date_Of_Consent_Site',\
             'Billing Trigger Date','WIP_Sites','Decommisioned_Site','Active_Sharing_Arrangement','Subsequent_Sharing_Arrangement','First_Active_Sharing_Deployment_Type',\
             'First_Active_Sharing_Start_Date','First_Active_Sharing_End_Date','BTS Site Acceptance Date','Billing Trigger Date','Date of Equipment Removal']

path='/content/TowerDB_Ireland_20210630 v1.xlsx'
sheet='TowerDB_Input_Ireland_20210630'
skiprows=0
skipcols=0
"""A coluna Infrastructure Ready Date contem 0 e deve ser feito replace"""
dates_tw = ['Infrastructure Ready Date']
#(path, sheetname, n_skiprows, n_skip_columns, site_index, cols_number, col_dates,
towerdb = read_files(path, sheet, skiprows, skipcols, 'Site ID ', dates_tw, bill_cols)
#
#towerdb['Billing Trigger Date'] = towerdb['Infrastructure Ready Date']

# Read CZ towerdb files
"""Merge to get DOER column"""
msa = pd.read_csv('/content/TowerDB_Ireland_20210731.csv', engine='python')
doer_col = msa[['Site ID ', 'Date of Equipment Removal']]
towerdb = towerdb.merge(doer_col, on='Site ID ')

towerdb.rename(columns={'Billing Trigger Date': 'Billing Trigger Date2'}, inplace=True)
towerdb['Billing Trigger Date'] = towerdb['Infrastructure Ready Date'].values
towerdb = towerdb[tw_cols]
towerdb.head()


In [76]:
towerdb.to_csv('/content/TowerDB_Ireland_20210831.csv', index=False)

In [28]:
"""Check columns received"""
def check_columns_received(df, bill_cols):
    twdb_col = list(df.columns)
    col_miss = [i for i in bill_cols if i not in twdb_col]

    if col_miss:
        df_col_missing = pd.DataFrame(col_miss, columns=['Column(s) Missing'])
        return df_col_missing
    else:
        print('\nNo missing columns.')

df_col_missing = check_columns_received(towerdb, tw_cols)
df_col_missing
#Tem error 3 columns faltando


No missing columns.


In [34]:
picklist = {
    "Strategic": ['Yes', 'No'],
    'Critical': ['Yes', 'No'],
    'Site Type (See structure Type)': ["DAS passive","GBT","RTT"],
    'INTP ': ["Standard","Core","Transmission"],
    'Core Site Type': ['Non Core', 'Case A', 'Case B'],
    'Transmission System': ["Macro","Public DAS","Long-term Mobile","Transmission"],
    'Macro Site - Transmission Hub Site': ['Yes', 'No'],
    'Macro Site - Transmission Hub Site with/without Shelters': ["With shelters","Without shelters","Non Transmission Hub Site"],
    'Room Configuration': ['Indoor','Outdoor', "KN type Outdoor","RBS type Outdoor"],
    'Power Supply': ['AC','DC', 'AC & DC','No Power'],
    'Air Conditioning': ['No','Yes; Indoor Air Conditioning','Yes; Indoor Free Air cooling / Free cooling units'],
    'Site Status': ['In Service','Ordered','Pipeline','Planned','To be dismantled'],
    'Strategic_Site_Bucket': ['Yes - 0-5%','Yes - 5-10%','Non Strategic'],
    'CriticalSite_Beyond_10': ['Beyond 10%','Within 10%','Non Critical'],
    'Sites_As_Metered_Estimated': ['Estimated Model','Metered Model'],
    'Meter_Sharing_Site': ['Yes', 'No'],
    "Bts_Sites": ['Yes', 'No'],
    'Phase1_Site_Or_Consent_Site': ['MSA (Phase 1)','PMA (Phase 2)'],
    'WIP_Sites': ['Yes', 'No'],
    'Active_Sharing_Arrangement': ['MORAN (On VF equipment)','MORAN (On non-VF equipment)','MOCN with Spectrum Pooling','Partial Active-Passive','No Active Sharing'],
    'Subsequent_Sharing_Arrangement': ['Yes', 'No']

}
"""Picklist check"""
actives = towerdb[towerdb['Site Status']=='In Service']
col_pick =['Site ID ',"Strategic",'Critical','Site Type (See structure Type)','INTP ','Core Site Type','Transmission System',\
    'Macro Site - Transmission Hub Site','Macro Site - Transmission Hub Site with/without Shelters',\
    'Room Configuration','Power Supply','Air Conditioning','Site Status','Strategic_Site_Bucket','CriticalSite_Beyond_10',\
    'Sites_As_Metered_Estimated','Meter_Sharing_Site',"Bts_Sites",'Phase1_Site_Or_Consent_Site','WIP_Sites','Active_Sharing_Arrangement',\
    'Subsequent_Sharing_Arrangement']
"""Picklist check"""
actives = towerdb[towerdb['Site Status']=='In Service']
df_picklist = check_picklist(actives, 'Site ID ','Site Status', col_pick, picklist)
df_picklist

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:248: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,Site ID,Site Status,Subsequent_Sharing_Arrangement
0,WH032,In Service,Blank Value
1,DN304,In Service,Blank Value
2,DN947,In Service,Blank Value
3,DX252,In Service,Blank Value


In [26]:
"""MoM check sites"""
df_mom = check_mom_bts(towerdb, 'Site ID ', "Bts_Sites", msa ,'Site ID ', "Bts_Sites")
df_mom

,Site ID,Bts_Sites


In [ ]:
""" Check On air Sites"""
def check_on_air_sites(df, df_index, df_cols):

    df_check = df[df_cols]
    df_check['sites'] = df_check[df_index]
    df_check = df_check.set_index('sites')
    #df_check = replace_values(df_check, df_cols, 0)

    df_errors = pd.DataFrame(columns=df_cols)

    for site in df[df_index]:
        for column in df_cols: 
            if df_check.loc[site,df_index] not in df_errors.index:
                df_errors.loc[site,df_index] = df_check.loc[site,df_index]
                if df_check.loc[site,column] == 0:
                    df_errors.loc[site,column] = 'Incorret picklist value or Blank Value'
            else:
                if df_check.loc[site,column] == 0:
                    df_errors.loc[site,column] = 'Incorret picklist value or Blank Value'
    df = df_errors[df_errors.iloc[:,1:]]
    df = df.dropna(how='all', axis=1).fillna('Ok')       
    #df = df_errors.dropna(how='all', axis=1)   
    return df

on_cols = ['Site ID ','Transmission System', 'Site Type (See structure Type)', 'Room Configuration', 'Air Conditioning',"Bts_Sites", 'Sites_As_Metered_Estimated']
df_on_air = check_on_air_sites(actives, 'Site ID ', on_cols)
df_on_air
#No errros

In [35]:
path_uip = '/content/UserInput_Ireland_July 21_06.07.2021.xlsx'
uip_names = ['Site_ID', 'BTS site applicable charge (Annual)', 'Commercials for sites beyond 10% cap of critical sites (Annual)']
uip = pd.read_excel(path_uip ,sheet_name='SiteLevel',usecols=[0,1,2],skiprows=2)
uip.columns = uip_names

msa_sites = [i for i in msa['Site ID ']]
tw_sites = [i for i in towerdb ['Site ID ']]
uip_sites = [i for i in uip['Site_ID']]


In [46]:
""" BTS sites"""
def check_new_sites(df_towerdb, tw_index, bts_col, bill_col, status_col, msa_list, towerdb_list, uip_list):
    
    # capture current date as string
    current_date = pd.to_datetime('now').date()
    # convert current to timestamp
    current_date = pd.to_datetime(current_date)
    
    filtered = df_towerdb[[tw_index, status_col]]

    #Finding for ALL NEW SITES
    new_site = [i for i in towerdb_list if i not in msa_list]
    new_sites = pd.DataFrame(new_site, columns=['New_Sites'])
    new_sites = pd.merge(new_sites, filtered, how='left', left_on=['New_Sites'], right_on=tw_index)
    new_sites = new_sites[['New_Sites', status_col]]

    
    #list of new sites out of UIP sheet
    bts_out_uis = [i for i in df_towerdb[(df_towerdb[bts_col]=='Yes')&(df_towerdb[status_col]=='In Service')][tw_index] if i not in uip_list]
    bts_out_uis = pd.DataFrame(bts_out_uis, columns=['Bts_Sites_Out_UIS_File'])
    bts_out_uis = pd.merge(bts_out_uis, filtered, how='left', left_on=['Bts_Sites_Out_UIS_File'], right_on=tw_index)
    bts_out_uis = bts_out_uis[['Bts_Sites_Out_UIS_File', status_col]]

    #create a copy to make index modifications
    df = df_towerdb.copy()

    #New columns with Site Codes
    df['Sites'] = df[tw_index]
    #defining site code to index
    df.set_index('Sites', inplace=True)

    #filtering by new sites
    df = df[df[tw_index].isin(new_site)]

    # Save information os sites with demerged date more than current date
    print(df[bill_col])
    df[bill_col] = df[bill_col].astype('datetime64[s]')
    df_site_bts = df[(df[bts_col]=='Yes') | (df[bill_col] > current_date)]
    
    #if not (new_sites.empty or bts_out_uis.empty or df_site_bts.empty):
    return new_sites, bts_out_uis, df_site_bts[[tw_index,status_col, bts_col, bill_col]]

msa_sites = [i for i in msa['Site ID ']]
tw_sites = [i for i in towerdb ['Site ID ']]
uip_sites = [i for i in uip['Site_ID']]
#(df_towerdb, tw_index, bts_col, bill_col, status_col, msa_list, towerdb_list, uip_list)
new_sites, bts_out_uis, df_site_bts = check_new_sites(towerdb, 'Site ID ', "Bts_Sites", \
                                                      'Billing Trigger Date','Site Status', msa_sites, tw_sites, uip_sites)
#no errors

Series([], Name: Billing Trigger Date, dtype: object)


In [52]:
"""BTS Wip"""
wip_out_tw_list, df_wip_and_bts_flagged = check_wip(actives,'Site ID ','WIP_Sites', "Bts_Sites", msa, 'Site ID ', "Bts_Sites")
# No errors

In [ ]:
"""Decomissioned Sites"""
def check_decommissioned(df,df_index, decom_col, doer_col):
    #c = country.lower()
    filtered = df[(df[decom_col]=='Yes')&(df[doer_col]=="")]
    return filtered[[df_index, decom_col, doer_col]]

df_decom = check_decommissioned(actives,'Site ID ','Decommisioned_Site', 'Date of Equipment Removal')
df_decom
# No errors

In [52]:
"""Check doer in service sites"""
def check_tw_bill_doer(df_tw, tw_index, date_col, status_col, status, type_col):
    
    t = type_col.lower()
    # capture current date as string
    current_date = pd.to_datetime('now').date()
    # convert current to timestamp
    current_date = pd.to_datetime(current_date)
    if not df_tw[date_col].empty:
        if t == 'doer':
            filtered = df_tw[(df_tw[status_col]==status)&(df_tw[date_col].astype('datetime64[ns]') < current_date)]
            return filtered[[tw_index, status_col, date_col]] 
        else:
            filtered = df_tw[(df_tw[status_col]==status)&(df_tw[date_col].astype('datetime64[ns]').empty)]
            return filtered[[tw_index, status_col, date_col]] 
    else:
        print('Nothing wrong in services sites!')
#check_tw_bill_doer(df_tw, tw_index, date_col, status_col, status, type_col)       
df_doer = check_tw_bill_doer(actives, 'Site ID ', 'Date of Equipment Removal','Site Status','In Service', 'doer')
df_doer
# No errors

,Site ID,Site Status,Date of Equipment Removal


In [72]:
def check_uip_tw(df_tw,tw_index, status_tw_col, decom_col, tw_bts_col, tw_critical_col, df_uip, uip_sites):

    filtered = df_tw[[tw_index, status_tw_col]]
    #tw_active = df_tw[df_tw['Site Status']=='In Service']
    count_tw_sites = [i for i in df_tw[df_tw[status_tw_col] =='In Service'][tw_index]]

    # check number of sites that are in uip file and doesn't have in df_tw
    uis_sites_not_in_towerdb = []
    #if not set(count_tw_sites).intersection(uip_sites):
    uis_sites_not_in_towerdb = [i for i in uip_sites if i not in count_tw_sites]
    if uis_sites_not_in_towerdb:
        uis_sites_not_in_towerdb = pd.DataFrame(uis_sites_not_in_towerdb,columns=['UIS In Month not active in TowerDB!'])
        uis_sites_not_in_towerdb = pd.merge(uis_sites_not_in_towerdb, filtered, how='left', left_on='UIS In Month not active in TowerDB!',\
                                        right_on=tw_index)
        uis_sites_not_in_towerdb = uis_sites_not_in_towerdb[['UIS In Month not active in TowerDB!', status_tw_col]]
    
    in_service_not_in_uis = [i for i in count_tw_sites if i not in uip_sites]
    in_service_not_in_uis = pd.DataFrame(in_service_not_in_uis,columns=['TowerDB Sites out of UIS In Month!'])
    in_service_not_in_uis = pd.merge(in_service_not_in_uis, filtered, how='left', left_on='TowerDB Sites out of UIS In Month!',\
                                    right_on=tw_index)
    in_service_not_in_uis = in_service_not_in_uis[['TowerDB Sites out of UIS In Month!', status_tw_col]]
    #check for decomissioned site not in uip files

    if decom_col != "":
        tw_decomiss = [i for i in df_tw[df_tw[decom_col]=='Yes'][tw_index] if i in uip_sites]
        decomiss_sites_in_uip = pd.DataFrame(tw_decomiss, columns=['Decomissioned Site in UIS File'])
        decomiss_sites_in_uip = pd.merge(decomiss_sites_in_uip, filtered, how='left', left_on='Decomissioned Site in UIS File',\
                                        right_on=tw_index)
        decomiss_sites_in_uip = decomiss_sites_in_uip[['Decomissioned Site in UIS File', status_tw_col]]
    
        #Check BTS sites
        bts_sites = [i for i in df_tw[df_tw[tw_bts_col]=='Yes'][tw_index]]
        bts_sites_out_uip = []
        if not set(bts_sites).intersection(uip_sites):
            bts_sites_out_uip = [i for i in bts_sites if i not in uip_sites]
        bts_sites_out_uip = pd.DataFrame(bts_sites_out_uip, columns=['BTS Site not in UIS File'])
        bts_sites_out_uip = pd.merge(bts_sites_out_uip, filtered, how='left', left_on='BTS Site not in UIS File',\
                                        right_on=tw_index)
        bts_sites_out_uip = bts_sites_out_uip[['BTS Site not in UIS File', status_tw_col]]

        #  Check for UIP critical sites 
        uip_critical = [i for i in df_uip[(df_uip['Commercials for sites beyond 10% cap of critical sites (Annual)']!='')&\
                                    df_uip['BTS site applicable charge (Annual)']!=""]['Site_ID']]
        bts_tw_critical = df_tw[df_tw[tw_critical_col]=='Beyond 10%'][tw_index]
        
        critical = []
        if len(bts_tw_critical) > 0:
            if set(uip_critical).intersection(bts_tw_critical):
                critical = [i for i in bts_tw_critical if i not in uip_critical]
                #print(critical)
        if critical:
            critical = pd.DataFrame(critical, columns=['Sites with critical level beyond 10% in out UIS File'])
            critical = pd.merge(critical, filtered, how='left', left_on='Sites with critical level beyond 10% in out UIS File',\
                                        right_on=tw_index)
            critical = critical[['Sites with critical level beyond 10% in out UIPS File', status_tw_col]]
        #if not (in_service_uip_sites.empty or decomiss_sites_in_uip.empty or bts_sites_out_uip.empty or critical.empty):
        return uis_sites_not_in_towerdb, in_service_not_in_uis, decomiss_sites_in_uip, bts_sites_out_uip, critical
    else:
        #Check BTS sites
        bts_sites = [i for i in df_tw[df_tw[tw_bts_col]=='Yes'][tw_index]]
        bts_sites_out_uip = []
        if not set(bts_sites).intersection(uip_sites):
            bts_sites_out_uip = [i for i in bts_sites if i not in uip_sites]
        bts_sites_out_uip = pd.DataFrame(bts_sites_out_uip, columns=['BTS Site not in UIS File'])
        bts_sites_out_uip = pd.merge(bts_sites_out_uip, filtered, how='left', left_on='BTS Site not in UIS File',\
                                        right_on=tw_index)
        bts_sites_out_uip = bts_sites_out_uip[['BTS Site not in UIS File', status_tw_col]]

        #  Check for UIP critical sites 
        uip_critical = [i for i in df_uip[(df_uip['Commercials for sites beyond 10% cap of critical sites (Annual)']!='')&\
                                    df_uip['BTS site applicable charge (Annual)']!=""]['Site_ID']]
        bts_tw_critical = df_tw[df_tw[tw_critical_col]=='Beyond 10%'][tw_index]
        critical = []
        #if len(uip_critical) > 0:
        critical = [i for i in bts_tw_critical if i not in uip_critical]
        critical = pd.DataFrame(critical, columns=['Sites with critical level beyond 10% in out UIS File'])
        critical = pd.merge(critical, filtered, how='left', left_on='Sites with critical level beyond 10% in out UIS File',\
                                        right_on=tw_index)
        critical = critical[['Sites with critical level beyond 10% in out UIS File', status_tw_col]]
        #if not (in_service_uip_sites.empty or bts_sites_out_uip.empty or critical.empty):
        return in_service_uip_sites, bts_sites_out_uip

"""Check UIP sites matches with Towerdb sites"""
uis_sites_not_in_towerdb, in_service_not_in_uis, decomiss_sites_in_uip, bts_sites_out_uip, critical = check_uip_tw(towerdb,'Site ID ', 'Site Status', \
                                                              'Decommisioned_Site', "Bts_Sites",'CriticalSite_Beyond_10', \
                                                              uip, uip_sites)

In [73]:
uis_sites_not_in_towerdb

,UIS In Month not active in TowerDB!,Site Status
0,DN354,Dismantled
1,RN038,Dismantled


In [74]:
in_service_not_in_uis

,TowerDB Sites out of UIS In Month!,Site Status
0,DN433,In Service
1,DN822,In Service
2,DNTWR,In Service
3,DN320,In Service
4,DNCF1,In Service
5,DNNGE,In Service
6,WH032,In Service
7,DN304,In Service


In [79]:
"""Check UIP sites matches with Towerdb sites"""
def check_uip_tw(df_tw,tw_index, status_tw_col, decom_col, tw_bts_col, tw_critical_col, df_uip, uip_sites):

    filtered = df_tw[[tw_index, status_tw_col]]
        #tw_active = df_tw[df_tw['Site Status']=='In Service']
    count_tw_sites = [i for i in df_tw[df_tw[status_tw_col] =='In Service'][tw_index]]

    # check number of sites that are in uip file and doesn't have in df_tw
    uis_sites_not_in_towerdb = []
    #if not set(count_tw_sites).intersection(uip_sites):
    uis_sites_not_in_towerdb = [i for i in uip_sites if i not in count_tw_sites]
    if uis_sites_not_in_towerdb:
        uis_sites_not_in_towerdb = pd.DataFrame(uis_sites_not_in_towerdb,columns=['UIS Sites True Up not active in TowerDB!'])
        uis_sites_not_in_towerdb = pd.merge(uis_sites_not_in_towerdb, filtered, how='left', left_on='UIS Sites True Up not active in TowerDB!',\
                                        right_on=tw_index)
        uis_sites_not_in_towerdb = uis_sites_not_in_towerdb[['UIS Sites True Up not active in TowerDB!', status_tw_col]]
    
    in_service_not_in_uis = [i for i in count_tw_sites if i not in uip_sites]
    in_service_not_in_uis = pd.DataFrame(in_service_not_in_uis,columns=['TowerDB Sites out of UIS True Up Sites!'])
    in_service_not_in_uis = pd.merge(in_service_not_in_uis, filtered, how='left', left_on='TowerDB Sites out of UIS True Up Sites!',\
                                    right_on=tw_index)
    in_service_not_in_uis = in_service_not_in_uis[['TowerDB Sites out of UIS True Up Sites!', status_tw_col]]
    #check for decomissioned site not in uip files

    if decom_col != "":
        tw_decomiss = [i for i in df_tw[df_tw[decom_col]=='Yes'][tw_index] if i in uip_sites]
        decomiss_sites_in_uip = pd.DataFrame(tw_decomiss, columns=['Decomissioned Site in UIS File'])
        decomiss_sites_in_uip = pd.merge(decomiss_sites_in_uip, filtered, how='left', left_on='Decomissioned Site in UIS File',\
                                        right_on=tw_index)
        decomiss_sites_in_uip = decomiss_sites_in_uip[['Decomissioned Site in UIS File', status_tw_col]]
    
        #Check BTS sites
        bts_sites = [i for i in df_tw[df_tw[tw_bts_col]=='Yes'][tw_index]]
        bts_sites_out_uip = []
        if not set(bts_sites).intersection(uip_sites):
            bts_sites_out_uip = [i for i in bts_sites if i not in uip_sites]
        bts_sites_out_uip = pd.DataFrame(bts_sites_out_uip, columns=['BTS Site not in UIS File'])
        bts_sites_out_uip = pd.merge(bts_sites_out_uip, filtered, how='left', left_on='BTS Site not in UIS File',\
                                        right_on=tw_index)
        bts_sites_out_uip = bts_sites_out_uip[['BTS Site not in UIS File', status_tw_col]]

        #  Check for UIP critical sites 
        uip_critical = [i for i in df_uip[(df_uip['Commercials for sites beyond 10% cap of critical sites (Annual)']!='')&\
                                    df_uip['BTS site applicable charge (Annual)']!=""]['Site_ID']]
        bts_tw_critical = df_tw[df_tw[tw_critical_col]=='Beyond 10%'][tw_index]
        
        critical = []
        if len(bts_tw_critical) > 0:
            if set(uip_critical).intersection(bts_tw_critical):
                critical = [i for i in bts_tw_critical if i not in uip_critical]
                #print(critical)
        if critical:
            critical = pd.DataFrame(critical, columns=['Sites with critical level beyond 10% in out UIS File'])
            critical = pd.merge(critical, filtered, how='left', left_on='Sites with critical level beyond 10% in out UIS File',\
                                        right_on=tw_index)
            critical = critical[['Sites with critical level beyond 10% in out UIPS File', status_tw_col]]
        #if not (in_service_uip_sites.empty or decomiss_sites_in_uip.empty or bts_sites_out_uip.empty or critical.empty):
        return uis_sites_not_in_towerdb, in_service_not_in_uis, decomiss_sites_in_uip, bts_sites_out_uip, critical
    else:
        #Check BTS sites
        bts_sites = [i for i in df_tw[df_tw[tw_bts_col]=='Yes'][tw_index]]
        bts_sites_out_uip = []
        if not set(bts_sites).intersection(uip_sites):
            bts_sites_out_uip = [i for i in bts_sites if i not in uip_sites]
        bts_sites_out_uip = pd.DataFrame(bts_sites_out_uip, columns=['BTS Site not in UIS File'])
        bts_sites_out_uip = pd.merge(bts_sites_out_uip, filtered, how='left', left_on='BTS Site not in UIS File',\
                                        right_on=tw_index)
        bts_sites_out_uip = bts_sites_out_uip[['BTS Site not in UIS File', status_tw_col]]

        #  Check for UIP critical sites 
        uip_critical = [i for i in df_uip[(df_uip['Commercials for sites beyond 10% cap of critical sites (Annual)']!='')&\
                                    df_uip['BTS site applicable charge (Annual)']!=""]['Site_ID']]
        bts_tw_critical = df_tw[df_tw[tw_critical_col]=='Beyond 10%'][tw_index]
        critical = []
        if len(uip_critical) > 0:
            if set(uip_critical).intersection(bts_tw_critical):
                critical = [i for i in bts_tw_critical if i not in uip_critical]
        critical = pd.DataFrame(critical, columns=['Sites with critical level beyond 10% in out UIS File'])
        critical = pd.merge(critical, filtered, how='left', left_on='Sites with critical level beyond 10% in out UIS File',\
                                        right_on=tw_index)
        critical = critical[['Sites with critical level beyond 10% in out UIS File', status_tw_col]]
        #if not (in_service_uip_sites.empty or bts_sites_out_uip.empty or critical.empty):
        return in_service_uip_sites, bts_sites_out_uip, critical


path_uiptrue = '/content/UserInput_Ireland_20210630 June delta.xlsx'
uip_names = ['Site_ID', 'BTS site applicable charge (Annual)', 'Commercials for sites beyond 10% cap of critical sites (Annual)']
uip_true = pd.read_excel(path_uip ,sheet_name='SiteLevel',usecols=[0,1,2],skiprows=2)
uip_true.columns = uip_names
uip_sites_true = [i for i in uip_true['Site_ID']]

uis_true_not_in_towerdb,in_service_uis_true, decomiss_in_uis, bts_sites_out_uis_true, critical_true = check_uip_tw(towerdb,'Site ID ', 'Site Status', \
                                                                'Decommisioned_Site', "Bts_Sites",'CriticalSite_Beyond_10', uip, uip_sites)
#No errors


In [80]:
uis_true_not_in_towerdb

,UIS Sites True Up not active in TowerDB!,Site Status
0,DN354,Dismantled
1,RN038,Dismantled


In [81]:
in_service_uis_true

,TowerDB Sites out of UIS True Up Sites!,Site Status
0,DN433,In Service
1,DN822,In Service
2,DNTWR,In Service
3,DN320,In Service
4,DNCF1,In Service
5,DNNGE,In Service
6,WH032,In Service
7,DN304,In Service


In [86]:
def check_diffs_v2(path_current, path_last, sheet='Commercial'):
    
    def lower_str(columns):
        newlist = list(map(lambda x: x.lower(), columns))
        return newlist

    def highlight_diff(data, color='yellow'):
        attr = 'background-color: {}'.format(color)
        other = data.xs('Current', axis='columns', level=-1)
        return pd.DataFrame(np.where(data.ne(other, level=0), attr, ''),
                            index=data.index, columns=data.columns)
    #type_file = type_file.lower()
    #if type_file =='excel':
    _actual = pd.read_excel(path_current,sheet_name=sheet).fillna('')
    _before = pd.read_excel(path_last,sheet_name=sheet).fillna('')

    df_all = pd.concat([_actual, _before],axis='columns', keys=['Current', 'Last'])
    df_final = df_all.swaplevel(axis='columns')[_actual.columns[1:]]

    #df_final.style.apply(highlight_diff, axis=None)
    if not df_final.empty:
        return df_final[(_actual != _before).any(1)].style.apply(highlight_diff, axis=None)
    else:
        print('\nNo differences Founded!\n')
path_before = '/content/UserInput_Ireland_20210731.xlsx'
path_curr = '/content/UserInput_Ireland_July 21_06.07.2021.xlsx'
df_comm = check_diffs_v2(path_curr, path_before)
df_comm
# No errors

In [88]:
def check_commercial(path_current, path_before, col_replace, col_names, merge_cols, col_order):
    
    def replace_values(df, columns, value=""):
        """
        Está voltando para float
        """
        invalid_values = ['N/A', 'n/a',"0", '-', '_', np.nan,'nan']
        #lista = []

        df.fillna("", inplace=True)
        #df[column] = df[column].astype('int64')

        for column in columns:
            lista = []
            for index in df[column]:
                #print(f"{column} -> {index}")
                if index in invalid_values:
                    lista.append(value)
                else:
                    lista.append(index)
            df[column] = lista

        return df

    def check_uip_commercial_values(df_actual, df_before, merge_cols):
        df_atual = uip_comercial_actual
        df_ant = uip_comercial_before
        df_cross = pd.merge(df_actual, df_before, on=merge_cols, \
                            how='left', suffixes=('_actual', '_before'), indicator='Exist')
        df_cross['Exist'] = np.where(df_cross.Exist == 'both', 'Yes', 'No')
        df_cross['Equal Values'] = df_cross['Exist']
        
        return df_cross
    # Check for commercial Values into current UIP File and compare with UIP File before
    uip_comercial_actual = pd.read_excel(path_current,sheet_name='Commercial', names=col_names)
    #uip_comercial_actual = uip_comercial_actual[['Charge_Type', 'Data_Type', 'Input_Value']]
    uip_comercial_actual = replace_values(uip_comercial_actual, col_replace, value=0)

    uip_comercial_before = pd.read_excel(path_before,sheet_name='Commercial', names=col_names )
    #uip_comercial_before = uip_comercial_before[['Charge_Type', 'Data_Type', 'Input_Value']]
    uip_comercial_before = replace_values(uip_comercial_before, col_replace, value=0)

    df_commercial =  check_uip_commercial_values(uip_comercial_actual, uip_comercial_before, merge_cols)

    #df_commercial = df_commercial.reindex(columns=col_order)
    df_commercial_diffs = df_commercial[df_commercial['Equal Values']=='No']
    if not df_commercial_diffs.empty:
        return df_commercial_diffs
    else:
        print('\nNo Errors founded!')

names = ['Charge_Type','Sub_Charge_Type', 'Param1','Param2', 'Data_Type', 'Input_Value',\
         'Description/Instruction', 'Frequency of Update']
merge_cols = ['Charge_Type','Sub_Charge_Type', 'Param1','Param2', 'Data_Type', \
              'Description/Instruction', 'Frequency of Update']
cols_ordered = ['Charge_Type','Sub_Charge_Type', 'Param1','Param2','Data_Type','Input_Value_actual',\
                'Input_Value_before','Equal Values','Description/Instruction', 'Frequency of Update']
# Check for commercial Values into current UIP File and compare with UIP File before

df_com = check_commercial(path_curr, path_before,['Input_Value'], names, merge_cols, cols_ordered)
df_com


No Errors founded!


Excel log

In [92]:
df_tw = [df_picklist, in_service_not_in_uis, uis_true_not_in_towerdb]
sheetnames_tw = ['Picklist Errors Actives Sites','TowerDB Sites out of UIS In Month!', "TowerDB Sites out of UIS True Up Sites", ""]

path_tw = '/content/towerdb_IE_errors.xlsx'
general_log_erros(df_tw, sheetnames_tw, path_tw)

/usr/local/lib/python3.7/dist-packages/openpyxl/workbook/child.py:102: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")


TA Input File

In [ ]:
#Comparando os ficheiros
pathta='/content/TowerDB_Ireland_20210531 1.xlsx'
pathold = '/content/TA_Input_Ireland_20210630.csv'
sheet='TA_Input_Ireland_20210531'
skiprows=0
skipcols=0
ta_save = '/content/TA_IE'
ta_bills = ['Site ID ','Customer Type ','Lease Start Date ', 'Lease End date ']
find_diffs_between_files(pathold, pathta, 'Site ID ', ta_bills,'',ta_save, 'mix','ta',sheet)


New Rows:     ['DL020 ', 'MNCVG']
Dropped Rows: []

Done.



In [72]:
#Read TA Input File
def read_files(path, sheetname,col_name, n_skiprows, n_skip_columns, site_index, cols_date, cols_int,\
               cols_amount, bill_cols, format='mix', type_date="%d/%m/%Y"):
    
    def replace_values(df, columns, value=""):
        """
        Está voltando para float
        """
        invalid_values = ['N/A', 'n/a',"0", '-', '_', np.nan,'nan', ' ', 'not available yet']

        for column in columns:
            lista = []
            df[column] = df[column].fillna(0)
            for index in df[column]:
                #print(f"{column} -> {index}")
                if index in invalid_values:
                    lista.append(value)
                else:
                    lista.append(index)
            df[column] = lista
        return df
    def date_parser(df, columns,format, type_date):
        for column in columns:
            if format == 'mix':
                df[column] = [date_obj.strftime(type_date) if not pd.isnull(date_obj) \
                            and not isinstance(date_obj, str) else date_obj for date_obj in df[column]]
                df[column] = df[column].astype(str)
            else:
                df[column] = [date_obj.strftime(type_date) if not pd.isnull(date_obj) else '' for date_obj in df[column]]
                df[column] = df[column].astype(str)
    #header=0, names = col_name,
    df = pd.read_excel(path, sheet_name = sheetname, engine='openpyxl', skiprows = n_skiprows)
    df = df.iloc[:,n_skip_columns:]
    df = df[col_name] 
    df = df.dropna(subset=[site_index], axis=0)
    df = replace_values(df, cols_amount, 0)
    for col in cols_amount:
        df[col] = df[col].fillna(0)
        df[col] = df[col].astype(int).apply(lambda x: f'{x:,}')
    df = replace_values(df, cols_int, 0)
    for col in cols_int:
        df[col] = df[col].fillna(0)
        df[col] = df[col].astype(int)
    df = df.fillna('')
    df = replace_values(df, bill_cols)
    date_parser(df, cols_date, format, type_date)
    # define the entiry columns which doesn't have NaN 
    return df
ta_col_ord = ["Site ID ","Site Name ","Address ","Site Provider","Site type",\
              "Tenant/Customer ","Customer Type "," Initial Rent  "," Current Rent  "," REC Uplift Charge ",\
              "Current Power Charge","Renegotiations ","Lease Start Date ","Lease End date ","Right to Renew ",\
              "End of Renewal Period ","Payment Terms ","Payment Frequency ","Rent Review ","Indexation Driver",\
              "Fixed Increase ","CPI ","Percentage","Increase frequency","Increase Date","VAT","Rate ","Termination date "]
interger = ['Increase frequency']
amount = [" Initial Rent  ", " Current Rent  "]
ta_bill_cols = ["Site ID ","Tenant/Customer ","Customer Type ",\
                "Lease Start Date ","Lease End date "]
ta_dates = ["Lease Start Date ","Lease End date ", "Increase Date"]
              
path='/content/TowerDB_Ireland_20210630 v1.xlsx'
sheet='TA_Input_Ireland_20210630'
skiprows=0
skipcols=0
#(path, sheetname,col_name, n_skiprows, n_skip_columns, site_index, cols_date, cols_int,\
               #cols_amount, bill_cols, format='mix', type_date="%d/%m/%Y")
ta = read_files(path, sheet,ta_col_ord, 0, 0, "Site ID ", ta_dates, interger,amount, ta_bill_cols)
ta.head(3)

In [78]:
ta.to_csv('/content/TA_Input_Ireland_20210831.csv', index=False)

In [ ]:
cols_date = ['Site ID ', 'Lease Start Date ', 'Lease End date ']
df_dates_erros = check_date_columns(ta, 'Site ID ', cols_date, 2)
# No errors

In [86]:
def check_lc_ta_dates(df,tw_index, start_date,end_date):

        filtered = df[pd.to_datetime(df[start_date]) > pd.to_datetime(df[end_date])]
        return filtered[[tw_index, start_date, end_date]]

df_ta_dates = check_lc_ta_dates(ta,"Site ID ", "Lease Start Date ","Lease End date ")
df_ta_dates

,Site ID,Lease Start Date,Lease End date


In [82]:
"""Picklist Values"""
picklist_ta = {
    'Customer Type ': ['MNO','OTMO']
}
ta['Customer Type '].value_counts()

MNO     494
OTMO    229
Name: Customer Type , dtype: int64

LC Input File

In [ ]:
#Comparando os ficheiros
pathlc='/content/TowerDB_Ireland_20210531 1.xlsx'
pathold_lc = '/content/LC_Input_Ireland_20210630.csv'
sheet_lc='LC_Input_Ireland_20210531'
skiprows=0
skipcols=0
lc_save = '/content/LC_IE'
lc_bills = ['Site ID ', ' Total Rent ', 'Lease Start Date ', 'Lease End date ']
find_diffs_between_files(pathold_lc, pathlc, 'Site ID ', lc_bills,'',lc_save, 'mix','ta',sheet_lc)

LC Validations

In [53]:
#Read TA Input File
def read_files(path, sheetname,col_name, n_skiprows, n_skip_columns, site_index, cols_date, cols_int,\
               cols_amount, bill_cols, format='mix', type_date="%d/%m/%Y"):
    
    def replace_values(df, columns, value=""):
        """
        Está voltando para float
        """
        invalid_values = ['N/A', 'n/a',"0", '-', '_', np.nan,'nan', ' ', 'not available yet']

        for column in columns:
            lista = []
            df[column] = df[column].fillna(0)
            for index in df[column]:
                #print(f"{column} -> {index}")
                if index in invalid_values:
                    lista.append(value)
                else:
                    lista.append(index)
            df[column] = lista
        return df
    def date_parser(df, columns,format, type_date):
        for column in columns:
            if format == 'mix':
                df[column] = [date_obj.strftime(type_date) if not pd.isnull(date_obj) \
                            and not isinstance(date_obj, str) else date_obj for date_obj in df[column]]
                df[column] = df[column].astype(str)
            else:
                df[column] = [date_obj.strftime(type_date) if not pd.isnull(date_obj) else '' for date_obj in df[column]]
                df[column] = df[column].astype(str)
    #header=0, names = col_name,
    df = pd.read_excel(path, sheet_name = sheetname, engine='openpyxl', skiprows = n_skiprows)
    df = df.iloc[:,n_skip_columns:]
    df = df[col_name] 
    df = df.dropna(subset=[site_index], axis=0)
    df = replace_values(df, cols_amount, 0)
    for col in cols_amount:
        df[col] = df[col].fillna(0)
        df[col] = df[col].astype(int).apply(lambda x: f'{x:,}')
    df = replace_values(df, cols_int, 0)
    for col in cols_int:
        df[col] = df[col].fillna(0)
        df[col] = df[col].astype(int)
    df = df.fillna('')
    df = replace_values(df, bill_cols)
    date_parser(df, cols_date, format, type_date)
    # define the entiry columns which doesn't have NaN 
    return df
lc_ord = ["Site ID ","Site Name ","Address ","Site Provider","Contract Type","Vendor"," Current Rent  ",\
          " Sublet/Uplift "," Total Rent ","Misc Fees ","Sublet Clause ","Conditions ","Uplift for Sublet ",\
          "Lease Start Date ","Lease End date ","Right to Renew ","End of Renewal Period ","Payment Terms ",\
          "Payment Frequency","Rent Review ","Fixed Increase ","Indexation Driver","CPI/Increase Type",\
          "Percentage","Increase Frequency","Increase date","VAT Subject","VAT","Termination date ","Observations"]
interger = [" Sublet/Uplift "]#'Increase frequency'
amount = []#" Current Rent  "," Total Rent "
ta_bill_cols = ["Site ID "," Total Rent ",\
                "Lease Start Date ","Lease End date "]
dates = ["Lease Start Date ","Lease End date ", "Increase date"]             
path='/content/TowerDB_Ireland_20210630 v1.xlsx'
sheet_lc='LC_Input_Ireland_20210630'
#(path, sheetname,col_name, n_skiprows, n_skip_columns, site_index, cols_date, cols_int,\
               #cols_amount, bill_cols, format='mix', type_date="%d/%m/%Y")
lc = read_files(path, sheet_lc,lc_ord, 0, 0, "Site ID ", dates, interger,amount, ta_bill_cols)
lc["Uplift for Sublet "] = [re.sub('^€', '', str(x)) for x in lc["Uplift for Sublet "]]
lc["Uplift for Sublet "] = lc["Uplift for Sublet "].replace('-', '')

lc.head()

In [77]:
lc.to_csv('/content/LC_Input_Ireland_20210831.csv', index=False)

In [85]:
def check_lc_ta_dates(df,tw_index, start_date,end_date):

        filtered = df[pd.to_datetime(df[start_date]) > pd.to_datetime(df[end_date])]
        return filtered[[tw_index, start_date, end_date]]

df_lc_dates = check_lc_ta_dates(lc,"Site ID ", "Lease Start Date ","Lease End date ")
df_lc_dates

,Site ID,Lease Start Date,Lease End date


In [ ]:
def check_amounts(df_check, df_index, columns, pattern=','):
    """
    Paramns:
    pattern: general (. to decimal), other (, to decimal)
    """

    df = df_check[columns]
    df['sites'] = df_check[df_index]
    df = df.set_index('sites')

    df_new = pd.DataFrame(columns=columns)
    
    df_duplicates = count_duplicates(df[df_index])
    if not df_duplicates.empty:
        df.drop_duplicates(subset=[df_index], inplace=True)

    for site in df[df_index]:
        for column in columns[1:]:
            print(df.loc[site,column])
            if not str(df.loc[site,column]).__contains__(pattern):
                #print(df.loc[site,column])
                if df.loc[site,df_index] not in df_new.index:
                    df_new.loc[site,df_index] = df.loc[site,df_index]
                    df_new.loc[site,column] = 'Incorret Format to separate decimal values'
                else:
                    df_new.loc[site,column] = 'Incorret Format to separate decimal values'

    df_new = df_new.dropna(how='all', axis=1).fillna('Ok')       
    if not df_new.empty:
        return df_new
    else: 
        print('No one columns with incorrect Amount format!')

"""Check Amount"""
lc_col_amounts = [' Total Rent ']
df_lc_amounts = check_amounts(lc, "Site ID ", lc_col_amounts, 2)
# No errors

In [ ]:
df_lc = [df_lc_dates_erros]
sheetnames_lc = ["Sites In Service with incorrect date"]

path_lc = '/content/lc_ie_errors.xlsx'
general_log_erros(df_lc, sheetnames_lc, path_lc)

/usr/local/lib/python3.7/dist-packages/openpyxl/workbook/child.py:102: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")
